## Auth and Token Service for MGP via API

The following workflow will show a user how to access the MGP API via the requests module for python. Specifically, this notebook will show a user how to:
* Get an access token
* Create a new API token
* List existing API tokens
* Delete existing API tokens

#### Fetching an access token

Firstly, the *requests* module needs to be imported to make API calls via python. If an import error occurs, the package will need to be installed onto you machine via the command *pip install requests*. We will also need the json module to interace with json data and to jsonify certain data later on

In [ ]:
import requests
import json

Here we will set our username and password as variables

In [ ]:
username = "your.email@address.com"
password= "yourpassword"

Now we will need to fetch a new token and assign it to a variable

In [ ]:
url = "https://account.maxar.com/auth/realms/mds/protocol/openid-connect/token"

payload = 'client_id=mgp&username={}&password={}&grant_type=password'.format(username, password)
headers = {'Content-Type': 'application/x-www-form-urlencoded'}

response = requests.request("POST", url, headers=headers, data=payload)

access_token = {'Content-Type': 'application/json', 'Authorization': 'Bearer {}'.format(response.json()['access_token'])}
print(access_token)

The previous call can be ran as many times as desired and will fetch a new token every time and set that token to the *access_token* variable. The token can be viewed by printing the *access_token* variable. A user can also refresh their existing token with the Refresh Token request shown below

In [ ]:
refresh_token = response.json()['refresh_token']

refresh_payload = 'grant_type=refresh_token&refresh_token={}&client_id=mgp'.format(refresh_token)

refresh_response = requests.request("POST", url, headers=headers, data=refresh_payload)

access_token = {'Content-Type': 'application/json', 'Authorization': 'Bearer {}'.format(response.json()['access_token'])}

#### Creating a new API token
An API token is different from an access token. An API token is a query parameter that will be passed into an MGP valid URL that will grant authorization for a user. This is especially convenient when utilizing third party apps such as QGIS

To create an API token, we will need an access token to authorize the intial call. We will be using the *access_token* variable from above in the headers of this call and the rest of the calls in this notebook

In [ ]:
create_token_url = "https://api.maxar.com/token-service/api/v1/token"

create_token_payload = json.dumps({"name": "Your token name", "description": "Your token description"})

create_token_response = requests.request("POST", create_token_url, headers=access_token, data=create_token_payload)

print(create_token_response.json())

new_token = create_token_response.json()['id']

With our newly created token, we can take the data from the *apiToken* variable and pass that into MGP requests to utilize auth in this way. We can list out all of our created tokens with the following request

In [ ]:
get_tokens_url = "https://api.maxar.com/token-service/api/v1/token"

get_tokens_response = requests.request("GET", get_tokens_url, headers=access_token)

print(get_tokens_response.json())

If for some reason a user wants to delete a token, they can pass the *id* of the token into the following call to delete it. Note that the *id* of the token is NOT the same as the *apiToken* variable that gets used for actual authorization

In [ ]:
delete_token_url = "https://api.maxar.com/token-service/api/v1/token/id/{}".format(new_token)

delete_token_response = requests.request("DELETE", delete_token_url, headers=access_token)

if delete_token_response.status_code == 200:
    print("Token {} successfully deleted".format(new_token))
else:
    print("Token {} not deleted, status code {} received".format(new_token, delete_token_response.status_code))